## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:

[[citizens_st_mobile]]

[[flats_dir_partner]]

[[entries_installation_points_dir_partner]]

[[installation_point_st_partner]]

[[companies_st_partner]]
___

In [10]:
query_text = """--sql
    SELECT
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.citizen_id AS citizen_id,
        citizens_st_mobile.trial_available AS trial_available,
        citizens_st_mobile.state AS state,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        flats_dir_partner.address_uuid AS address_uuid,
        entries_installation_points_dir_partner.city AS city,
        installation_point_st_partner.partner_uuid AS partner_uuid,
        installation_point_st_partner.monetization AS monetization,
        installation_point_st_partner.monetization_is_allowed AS monetization_is_allowed,
        flats_dir_partner.created_at AS flat_created_at,
        citizens_st_mobile.activated_at AS activated_at
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.flats_dir_partner_ch AS flats_dir_partner 
        ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points_dir_partner 
        ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
    LEFT JOIN db1.installation_point_st_partner_ch AS installation_point_st_partner
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
    LEFT JOIN db1.companies_st_partner_ch AS companies_st_partner
        ON installation_point_st_partner.partner_uuid = companies_st_partner.partner_uuid
        AND companies_st_partner.report_date = citizens_st_mobile.report_date
    WHERE citizens_st_mobile.state = 'activated'
    limit 10
    """

ch.get_schema(query_text)

(
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String,
    `city` String,
    `partner_uuid` String,
    `monetization` Int16,
    `monetization_is_allowed` Int16,
    `flat_created_at` DateTime,
    `activated_at` DateTime
)


In [13]:
query_text = """--sql
CREATE TABLE db1.rep_mobile_citizens_id_city_partner
(
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String,
    `city` String,
    `partner_uuid` String,
    `monetization` Int16,
    `monetization_is_allowed` Int16,
    `flat_created_at` DateTime,
    `activated_at` DateTime
)
ENGINE = MergeTree()
ORDER BY citizen_id
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [3]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.rep_mobile_citizens_id_city_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR 40 MINUTE TO db1.rep_mobile_citizens_id_city_partner AS
   SELECT
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.citizen_id AS citizen_id,
        citizens_st_mobile.trial_available AS trial_available,
        citizens_st_mobile.state AS state,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        flats_dir_partner.address_uuid AS address_uuid,
        entries_installation_points_dir_partner.city AS city,
        installation_point_st_partner.partner_uuid AS partner_uuid,
        installation_point_st_partner.monetization AS monetization,
        installation_point_st_partner.monetization_is_allowed AS monetization_is_allowed,
        flats_dir_partner.`created_at` AS flat_created_at,
        citizens_st_mobile_ch.activated_at AS activated_at
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.flats_dir_partner_ch AS flats_dir_partner 
        ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points_dir_partner 
        ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
    LEFT JOIN db1.installation_point_st_partner_ch AS installation_point_st_partner
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
    LEFT JOIN db1.companies_st_partner_ch AS companies_st_partner
        ON installation_point_st_partner.partner_uuid = companies_st_partner.partner_uuid
        AND companies_st_partner.report_date = citizens_st_mobile.report_date
    WHERE citizens_st_mobile.state = 'activated'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.rep_mobile_citizens_id_city_partner
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)


report_date,citizen_id,trial_available,state,flat_uuid,address_uuid,city,partner_uuid,monetization,monetization_is_allowed,flat_created_at,activated_at
date,i64,i64,str,str,str,str,str,i64,i64,datetime[ns],datetime[ns]
2025-09-02,1600,0,"""activated""","""a8d89bec-a777-4cb7-b664-d0fd1f…","""e485bf66-c156-43e6-8739-5acc63…","""Алматы""","""af8b5bf4-0108-4a0c-acc2-096f94…",0,0,2019-06-07 12:53:52,1970-01-01 00:00:00
2025-09-02,1601,0,"""activated""","""9b7e1da4-9f91-4cc9-86a5-3b3e47…","""e485bf66-c156-43e6-8739-5acc63…","""Алматы""","""af8b5bf4-0108-4a0c-acc2-096f94…",0,0,2019-06-07 13:03:32,1970-01-01 00:00:00
2025-09-02,1602,0,"""activated""","""a8d89bec-a777-4cb7-b664-d0fd1f…","""e485bf66-c156-43e6-8739-5acc63…","""Алматы""","""af8b5bf4-0108-4a0c-acc2-096f94…",0,0,2019-06-07 12:53:52,1970-01-01 00:00:00
2025-09-02,1750,0,"""activated""","""5b31916f-7e18-44e2-91fd-89ce88…","""ac85a32c-ff99-47a9-9200-e5c5c5…","""Тюмень""","""adeffb47-1707-4976-a4a6-e6eacf…",1,1,2019-06-30 15:34:02,1970-01-01 00:00:00
2025-09-02,1605,0,"""activated""","""18ae5d80-043e-4fcd-b269-1a81c0…","""e485bf66-c156-43e6-8739-5acc63…","""Алматы""","""af8b5bf4-0108-4a0c-acc2-096f94…",0,0,2019-06-07 07:31:08,1970-01-01 00:00:00
…,…,…,…,…,…,…,…,…,…,…,…
2025-09-02,1699,0,"""activated""","""""","""""","""""","""""",0,0,1970-01-01 00:00:00,1970-01-01 00:00:00
2025-09-02,1708,0,"""activated""","""14c41817-baae-4cfd-87f8-77592e…","""0a2f9718-be59-4265-a635-1f431e…","""Иннополис""","""adeffb47-1707-4976-a4a6-e6eacf…",0,0,2019-06-22 09:13:01,1970-01-01 00:00:00
2025-09-02,1707,0,"""activated""","""77239186-6e2c-4a86-aa36-cf449f…","""3e077a30-2917-4af8-8782-7ebaa1…","""Нур-Султан (Астана)""","""af8b5bf4-0108-4a0c-acc2-096f94…",0,0,2019-06-21 12:12:46,1970-01-01 00:00:00


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.rep_mobile_citizens_id_city_partner DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [2]:
query_text = """--sql
    DROP TABLE db1.rep_mobile_citizens_id_city_partner_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [12]:
query_text = """--sql
    DROP TABLE db1.rep_mobile_citizens_id_city_partner
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.rep_mobile_citizens_id_city_partner_mv
"""

ch.query_run(query_text)